# Libs

In [1]:
import os
# while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

d:\DANG NHI\repos\master-llm-rag-vnlaw


In [2]:
import os
import regex as re
import random
import json
import pandas as pd
import numpy as np

from itertools import combinations
from itertools import product

base_model_id = "FacebookAI/xlm-roberta-base"


# data_path = "/kaggle/input/vn-law-test/data"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 100

files = os.listdir(data_path)
data_stored = []

# Load tokenizer

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModel.from_pretrained(base_model_id)

# Chunking

In [5]:
# khi split chunking không quá quan tâm đến độ dài
# mỗi chunk phải là một đoạn hoặc một câu hoàn chỉnh
# nếu là bảng thì phải tách riêng

def preprocess_phrase_chunking(text):
    s = text
    # s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', s)
    # s = re.sub('Độc lập - Tự do - Hạnh phúc', '', s)
    s = re.sub('[- ]+', ' ', s)
    s = re.sub('[…]+', '…', s)
    s = s.strip()
    return s

def count_tokens(text):
    return len(tokenizer.encode(text))

def last_words(text, n=10):
    lst = text.split(" ")[-n:]
    return " ".join(lst)

In [ ]:
def chunking_long_normaltext(input_text, remain_text="", max_len=512, overlap=100):
    chunks = []
    words = input_text.split(" ")
    current_text = remain_text

    for word in words:
        current_text += " " + word
        if count_tokens(current_text) > max_len - 10:
            current_text = current_text.strip()
            chunks.append(current_text)
            j = 1
            while count_tokens(last_words(current_text, n=j)) < overlap - 10:
                j += 1
            current_text = " ".join(current_text.split(" ")[-j:])
            current_text = current_text.strip()
    if len(current_text) > 0:
        current_text = current_text.strip()    
        chunks.append(current_text)
    return chunks

def chunking_long_table(input_text, remain_text='', max_len=512, overlap=100):
    chunks = []
    rows = input_text.split("\n")
    current_text = remain_text
    if len(rows) < 5:
        chunks = chunking_long_normaltext(input_text, remain_text, max_len, overlap)
        return chunks

    counts = [count_tokens(r) for r in rows]
    counts[0] = counts[0] + count_tokens(remain_text)

    current_chunk_idx = []
    for i in range(len(counts)):
        if sum([counts[j] for j in current_chunk_idx]) + counts[i] < MAX_LEN - 10:
            current_chunk_idx.append(i)
        else:
            current_text += "\n".join([rows[j] for j in current_chunk_idx])
            current_text = current_text.strip() + "\n (bảng còn tiếp)"

            chunks.append(current_text)

            j = 1
            while sum([counts[x] for x in current_chunk_idx[-j:]]) < OVERLAP - 10:
                j += 1

            current_text = " \n (tiếp tục bảng) \n" + "\n".join([rows[j] for j in current_chunk_idx[-j:]]) + "\n"
            current_chunk_idx = current_chunk_idx[-j:] + [i]

    if len(current_chunk_idx) > 0:
        current_text += "\n".join([rows[j] for j in current_chunk_idx])
        chunks.append(current_text)
    return chunks

def chunking_long_text(input_text, remain_text='', max_len=512, overlap=100):
    if input_text.startswith("Bắt đầu bảng"):
        return chunking_long_table(input_text, remain_text, max_len, overlap)
    else:
        return chunking_long_normaltext(input_text, remain_text, max_len, overlap)


def token_chunking(content, max_len=512, overlap=100):
    chunks = []
    counts = [count_tokens(c) for c in content]

    curr_content = ""
    curr_group_idx = []

    for i in range(len(content)):
        if sum([counts[i] for i in curr_group_idx]) + counts[i] < MAX_LEN:
            curr_group_idx.append(i)
        else:
            curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
            curr_content = curr_content.strip()
            chunks.append(curr_content)

            # split if text i too long
            if counts[i] > MAX_LEN:
                curr_content = last_words(curr_content, n=OVERLAP//2)
                small_split = chunking_long_text(content[i], curr_content , MAX_LEN, OVERLAP)
                chunks += small_split
                curr_content = last_words(small_split[-1], n= 10)
                curr_group_idx = []
            else:
                j = 1
                while (sum([counts[i] for i in curr_group_idx[-j:]]) < OVERLAP) and j < len(curr_group_idx):
                    j += 1
                curr_group_idx = curr_group_idx[-j:] + [i]
                curr_content = ""
    if len(curr_group_idx) > 0:
        curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
        curr_content = curr_content.strip()
        chunks.append(curr_content)
    return chunks

### chunking all test

In [49]:
data_stored = []

field = []
# for file in files:

file = "NhaDat.json"
print(file)
with open(f"{data_path}/{file}", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

for data in raw_data:
    name = data['name']
    content = data['content']
    print((name, len(content)))
    for i in range(min(5, len(content))):
        content[i] = preprocess_phrase_chunking(content[i])

    counts = [count_tokens(c) for c in content]

    results = token_chunking(content, max_len=512, overlap=100)
    field.append(results)
data_stored.append(field)

NhaDat.json
('Quyết định 709/QĐ-UBND', 74)
('Nghị định 10/2023/NĐ-CP', 149)
('Quyết định 350/QĐ-UBND', 20)
('Quyết định 09/2023/QĐ-UBND', 21)
('Quyết định 692/QĐ-UBND', 57)
('Quyết định 869/QĐ-UBND', 301)
('Công văn 1347/BXD-QLN', 8)
('Công điện 02/CĐ-TCT', 11)
('Quyết định 10/2023/QĐ-UBND', 80)
('Công văn 1551/BXD-QLN', 35)
('Quyết định 719/QĐ-BTNMT', 52)
('Quyết định 291/QĐ-UBND', 38)
('Quyết định 20/2023/QĐ-UBND', 27)
('Quyết định 723/QĐ-UBND', 32)
('Quyết định 724/QĐ-UBND', 31)
('Quyết định 05/2023/QĐ-UBND', 49)
('Quyết định 1064/QĐ-UBND', 45)
('Nghị định 10/2023/NĐ-CP', 149)
('Quyết định 338/QĐ-TTg', 321)
('Công văn 200/TTg-NN', 9)
('Quyết định 645/QĐ-UBND', 31)
('Quyết định 08/2023/QĐ-UBND', 78)
('Quyết định 547/QĐ-UBND', 57)
('Quyết định 05/2023/QĐ-UBND', 56)
('Quyết định 609/QĐ-UBND', 26)
('Công văn 2635/BTC-QLCS', 35)
('Kế hoạch 905/KH-UBND', 85)
('Nghị quyết 03/2023/NQ-HĐND', 26)
('Quyết định 776/QĐ-UBND', 69)
('Quyết định 549/QĐ-UBND', 97)
('Quyết định 14/2023/QĐ-UBND', 133)

In [51]:
import json

# Convert data_stored to a JSON-serializable format if needed
# Save the data to a file
with open('data/data_stored.json', 'w', encoding='utf-8') as f:
    json.dump(data_stored, f, ensure_ascii=False, indent=2)

### Build and test

In [14]:
file = "GiaoDuc.json"
# target_doc = "Quyết định 4662/QĐ-BGDĐT"
target_doc = "Công văn 6133/BGDĐT-QLCL"

with open(f"{data_path}/{file}", "r", encoding="utf-8") as f:
    data = json.load(f)

for doc in data:
    if doc['name'] == target_doc:
        break   

In [17]:
table = doc['content'][1]

In [20]:
chunking_long_table(table, remain_text='', max_len=512, overlap=100)

KeyboardInterrupt: 

In [19]:
table.split("\n")

['Bắt đầu bảng: ',
 'Kính gửi: | - Các Sở Giáo dục và Đào tạo; - Các đại học, trường đại học có trường THPT Chuyên 1. Lịch thi a) Ngày 05/01/2024: Thi viết các môn Toán, Vật lí, Hóa học, Sinh học, Ngữ văn, Lịch sử, Địa lí, Tiếng Anh, Tiếng Nga, Tiếng Pháp, Tiếng Trung Quốc và thi lập trình trên máy vi tính môn Tin học. b) Ngày 06/01/2024: Thi viết các môn Toán, Vật lí, Hóa học, Sinh học; Thi nói các môn Tiếng Anh, Tiếng Nga, Tiếng Pháp, Tiếng Trung Quốc và thi lập trình trên máy vi tính môn Tin học. c) Giờ cắt bì đề thi tại phòng thi: 07 giờ 50 phút. Thời gian bắt đầu làm bài của mỗi buổi thi: 08 giờ 00 phút. 2. Nội dung thi Theo Chương trình giáo dục trung học phổ thông (THPT) 2006 và hướng dẫn thực hiện chương trình chuyên sâu các môn chuyên cấp THPT Công văn số 10803/BGDĐT-GDTrH ngày 16/12/2009 của Bộ GDĐT. 3. Bàn giao đề thi Cục Quản lý chất lượng (QLCL) sẽ thông báo địa điểm, thời gian bàn giao đề thi cho Lãnh đạo đơn vị dự thi nơi đặt Hội đồng coi thi theo chế độ mật. Trường hợp 

In [16]:
content = doc['content']
print(len(content))

chunks = []
counts = [count_tokens(c) for c in content]

2


In [12]:
content

['BỘ GIÁO DỤC VÀ ĐÀO TẠO \n Số: 6133/BGDĐT-QLCL V/v tổ chức Kỳ thi chọn HSG quốc gia THPT năm học 2023-2024 \n Hà Nội, ngày 03 tháng 11 năm 2023',
 'Bắt đầu bảng: \nKính gửi: | - Các Sở Giáo dục và Đào tạo; - Các đại học, trường đại học có trường THPT Chuyên 1. Lịch thi a) Ngày 05/01/2024: Thi viết các môn Toán, Vật lí, Hóa học, Sinh học, Ngữ văn, Lịch sử, Địa lí, Tiếng Anh, Tiếng Nga, Tiếng Pháp, Tiếng Trung Quốc và thi lập trình trên máy vi tính môn Tin học. b) Ngày 06/01/2024: Thi viết các môn Toán, Vật lí, Hóa học, Sinh học; Thi nói các môn Tiếng Anh, Tiếng Nga, Tiếng Pháp, Tiếng Trung Quốc và thi lập trình trên máy vi tính môn Tin học. c) Giờ cắt bì đề thi tại phòng thi: 07 giờ 50 phút. Thời gian bắt đầu làm bài của mỗi buổi thi: 08 giờ 00 phút. 2. Nội dung thi Theo Chương trình giáo dục trung học phổ thông (THPT) 2006 và hướng dẫn thực hiện chương trình chuyên sâu các môn chuyên cấp THPT Công văn số 10803/BGDĐT-GDTrH ngày 16/12/2009 của Bộ GDĐT. 3. Bàn giao đề thi Cục Quản lý chấ

In [9]:
token_chunking(content, max_len=512, overlap=100)

['BỘ GIÁO DỤC VÀ ĐÀO TẠO \n Số: 4662/QĐ-BGDĐT \n Hà Nội, ngày 29 tháng 12 năm 2023\nQUYẾT ĐỊNH\nVỀ VIỆC PHÊ DUYỆT MÔ HÌNH GIÁO DỤC LÝ TƯỞNG CÁCH MẠNG, ĐẠO ĐỨC, LỐI SỐNG CHO HỌC SINH, SINH VIÊN\nBỘ TRƯỞNG BỘ GIÁO DỤC VÀ ĐÀO TẠO\nCăn cứ Nghị định số 86/2022/NĐ-CP ngày 24/10/2022 của Chính phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Giáo dục và Đào tạo;\nCăn cứ Quyết định số 1895/QĐ-TTg ngày 11/11/2021 của Thủ tướng Chính phủ phê duyệt Chương trình “Tăng cường giáo dục lý tưởng cách mạng, đạo đức, lối sống và khơi dậy khát vọng cống hiến cho thanh niên, thiếu niên, nhi đồng giai đoạn 2021-2030”;\nCăn cứ Kế hoạch số 638/KH-BGDĐT ngày 18/04/2023 của Bộ trưởng Bộ Giáo dục và Đào tạo về việc khảo sát, đánh giá thực trạng và tổ chức thí điểm xây dựng mô hình giáo dục lý tưởng cách mạng, đạo đức, lối sống cho học sinh, sinh viên;\nCăn cứ Quyết định số 3863/QĐ-BGDĐT ngày 17/11/2023 của Bộ trưởng Bộ Giáo dục và Đào tạo về việc thành lập Hội đồng thẩm định Mô hình giáo dục

In [ ]:
curr_content = ""
curr_group_idx = []

for i in range(len(content)):
    if sum([counts[i] for i in curr_group_idx]) + counts[i] < MAX_LEN:
        curr_group_idx.append(i)
    else:
        curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
        curr_content = curr_content.strip()
        chunks.append(curr_content)
        # print("curr_content:", curr_content)

        # split if text i too long
        if counts[i] > MAX_LEN:
            curr_content = last_words(curr_content, n=OVERLAP//2)
            # if "Miếng băng gạc cho trái tim" in content[i]:
            #     break
            small_split = chunking_long_text(content[i], curr_content , MAX_LEN, OVERLAP)
                
            chunks += small_split
            # print("len(small_split):", len(small_split))
            curr_content = last_words(small_split[-1], n=20)
            curr_group_idx = []
        else:
            j = 1
            while (sum([counts[i] for i in curr_group_idx[-j:]]) < OVERLAP) and j < len(curr_group_idx):
                j += 1
            curr_group_idx = curr_group_idx[-j:] + [i]
            curr_content = ""
if len(curr_group_idx) > 0:
    curr_content = curr_content + "\n" + "\n".join([content[j] for j in curr_group_idx])
    curr_content = curr_content.strip()
    chunks.append(curr_content)

In [51]:
i = 155
print(content[i])

Bắt đầu bảng: 
Miếng băng gạc cho trái tim - Mẹ, mẹ đang làm gì thế? Cô bé Su chỉ mới 6 tuổi hỏi mẹ. - Mẹ đang nấu món thịt hầm cho cô Xuân hàng xóm. - Vì sao ạ? - Su thắc mắc hỏi. - Vì cô Xuân đang rất buồn con ạ. Con gái cô ấy vừa qua đời và trái tim cô ấy đang tan nát. Chúng ta sẽ chăm sóc cô ấy một thời gian. - Mẹ Su dịu dàng trả lời. - Tại sao lại thế hả mẹ? - Su vẫn chưa hiểu. - Thế này nhé con yêu, khi một người rất buồn, họ sẽ không thể làm tốt ngay cả những việc rất nhỏ như nấu bữa tối hay làm một số việc vặt khác. Vì chúng ta cùng sống trong một tòa chung cư và cô Xuân là hàng xóm của gia đình mình, chúng ta cần phải giúp đỡ cô ấy. Cô Xuân sẽ không bao giờ còn có thể nói chuyện, ôm hôn con gái cô ấy hoặc làm bất cứ điều gì như mẹ con mình có thể làm cùng nhau. Con là một cô bé thông minh, có thể con sẽ nghĩ ra cách nào đó để giúp đỡ cô ấy. Su suy nghĩ rất nghiêm túc về những điều mẹ nói và cố gắng tìm cách giúp cô Xuân đỡ đau buồn. Vài phút sau, Su đã ở trước căn hộ của cô Xu

In [56]:
max_len = MAX_LEN
overlap = OVERLAP

In [ ]:
chunks = []
words = content[i].split(" ")
current_text = curr_content

for word in words:
    current_text += " " + word
    if count_tokens(current_text) > max_len - 10:
        current_text = current_text.strip()
        chunks.append(current_text)
        j = 1
        while count_tokens(last_words(current_text, n=j)) < overlap - 10:
            j += 1
        current_text = " ".join(current_text.split(" ")[-j:])
        current_text = current_text.strip()
if len(current_text) > 0:
    current_text = current_text.strip()    
    chunks.append(current_text)

j: 73
j: 72


In [68]:
count_tokens(current_text)

469

In [ ]:
def chunking_long_normaltext(input_text, remain_text="", max_len=512, overlap=100):
    chunks = []
    words = input_text.split(" ")
    current_text = remain_text
    for word in words:
        current_text += " " + word
        if count_tokens(current_text) > max_len - 10:
            current_text = current_text.strip()
            chunks.append(current_text)

            j = 1
            while (count_tokens(" ".join(current_text.split(" ")[-j:])) < overlap - 10) and j < count_tokens(current_text):
                j += 1
            current_text = " ".join(words[-j:])
            current_text = current_text.strip()
    if len(current_text) > 0:
        current_text = current_text.strip()    
        chunks.append(current_text)
    return chunks

In [49]:
content[i]

'Bắt đầu bảng: \nMiếng băng gạc cho trái tim - Mẹ, mẹ đang làm gì thế? Cô bé Su chỉ mới 6 tuổi hỏi mẹ. - Mẹ đang nấu món thịt hầm cho cô Xuân hàng xóm. - Vì sao ạ? - Su thắc mắc hỏi. - Vì cô Xuân đang rất buồn con ạ. Con gái cô ấy vừa qua đời và trái tim cô ấy đang tan nát. Chúng ta sẽ chăm sóc cô ấy một thời gian. - Mẹ Su dịu dàng trả lời. - Tại sao lại thế hả mẹ? - Su vẫn chưa hiểu. - Thế này nhé con yêu, khi một người rất buồn, họ sẽ không thể làm tốt ngay cả những việc rất nhỏ như nấu bữa tối hay làm một số việc vặt khác. Vì chúng ta cùng sống trong một tòa chung cư và cô Xuân là hàng xóm của gia đình mình, chúng ta cần phải giúp đỡ cô ấy. Cô Xuân sẽ không bao giờ còn có thể nói chuyện, ôm hôn con gái cô ấy hoặc làm bất cứ điều gì như mẹ con mình có thể làm cùng nhau. Con là một cô bé thông minh, có thể con sẽ nghĩ ra cách nào đó để giúp đỡ cô ấy. Su suy nghĩ rất nghiêm túc về những điều mẹ nói và cố gắng tìm cách giúp cô Xuân đỡ đau buồn. Vài phút sau, Su đã ở trước căn hộ của cô 

In [43]:
mychunks = chunking_long_normaltext(content[i], remain_text= curr_content , max_len=512, overlap=100)

luận nhóm. HS cần được luân phiên nhau làm “nhóm trưởng”, “ thư kí” cũng như luân phiên đại diện cho nhóm để trình bày kết quả thảo luận.
Ví dụ:
Để giáo dục lòng nhân ái cho HS tiểu học, có thể tổ chức cho HS thảo luận câu chuyện sau:
- Cần tạo cơ hội cho HS có thể tham gia vào các nhóm khác nhau với các bạn khác nhau để các em có thể tương tác, học hỏi lẫn nhau.
- Các thành viên trong nhóm phải ngồi đối diện nhau khi trao đổi, chia sẻ, thảo luận.
- Các thành viên phải lắng nghe ý kiến của bạn trong nhóm và mạnh dạn chia sẻ suy nghĩ, ý tưởng của bản thân
- Mỗi người đều tuân theo sự điều khiển của nhóm trưởng.
- Nhiệm vụ thảo luận phải phù hợp với chủ đề bài học, với khả năng của HS, phù hợp với thời lượng và CSVC, trang thiết bị hiện có.
- Nhiệm vụ của mỗi nhóm có thể giống hoặc khác nhau.
- Cách trình bày kết quả hoạt động nhóm có thể theo nhiều hình thức (bằng lời, bằng tranh vẽ, bằng tiểu phẩm, bằng văn bản viết trên giấy, ...; có thể do một người thay mặt nhóm trình bày hoặc có th

In [ ]:
mychun

['luận nhóm. HS cần được luân phiên nhau làm “nhóm trưởng”, “ thư kí” cũng như luân phiên đại diện cho nhóm để trình bày kết quả thảo luận.\nVí dụ:\nĐể giáo dục lòng nhân ái cho HS tiểu học, có thể tổ chức cho HS thảo luận câu chuyện sau:\n- Cần tạo cơ hội cho HS có thể tham gia vào các nhóm khác nhau với các bạn khác nhau để các em có thể tương tác, học hỏi lẫn nhau.\n- Các thành viên trong nhóm phải ngồi đối diện nhau khi trao đổi, chia sẻ, thảo luận.\n- Các thành viên phải lắng nghe ý kiến của bạn trong nhóm và mạnh dạn chia sẻ suy nghĩ, ý tưởng của bản thân\n- Mỗi người đều tuân theo sự điều khiển của nhóm trưởng.\n- Nhiệm vụ thảo luận phải phù hợp với chủ đề bài học, với khả năng của HS, phù hợp với thời lượng và CSVC, trang thiết bị hiện có.\n- Nhiệm vụ của mỗi nhóm có thể giống hoặc khác nhau.\n- Cách trình bày kết quả hoạt động nhóm có thể theo nhiều hình thức (bằng lời, bằng tranh vẽ, bằng tiểu phẩm, bằng văn bản viết trên giấy, ...; có thể do một người thay mặt nhóm trình bày

# Pairing

In [53]:
pair_data = []
pairs_simularity_document_level = 0
for i in range(len(data_stored)):
    l = len(data_stored[i])
    pairs_simularity_document_level += (l*(l-1))//2
pairs_simularity_document_level

41905

In [54]:
def get_very_positive_pairs(data_stored, max_pairs= 500000):
    pairs = []
    for i in range(len(data_stored)):
        for j in range(len(data_stored[i])):
            chunks = data_stored[i][j]
            for k in range(len(chunks)):
                for l in range(k+1, len(chunks)):
                    pairs.append(((i,j,k), (i,j,l), 1))
    return random.sample(pairs, max_pairs)

very_positive_pairs = get_very_positive_pairs(data_stored)
len(very_positive_pairs)

500000

# Embeddings

In [ ]:
def embedding_preprocess(text):
    s = text
    s = re.sub(r"\|", " ", s)
    # s = re.sub(r"[\d\.-]+", "number", s)
    s = re.sub(r"\d[\d,.A-Za-x]{2,}", '', s)
    # s = re.sub(r'\b\d+(?:[,.]\d+)?%?\b|\b\d+[A-Za-z]\d*(?:\.\d+)*\b', 'number', s)
    s = s.strip()
    return s

def embedding(text):
    input_ids = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model(**input_ids)
    return output

embed = embedding_preprocess(table_text)
embed = embedding(embed)

In [83]:
(count_tokens(table_text), count_tokens(embed))

(62119, 32121)

In [21]:
tokens = tokenizer.encode_plus(
    texts[7], 
    truncation=True, 
    padding='max_length',
    return_tensors='pt'
    )
tokens['input_ids'].shape


NameError: name 'texts' is not defined

In [ ]:
# def tokenizer_chunking(content, tokenizer, max_len=512, overlap=50):
#     tokens = tokenizer.encode(content, add_special_tokens=False)
#     chunks = []
#     start = 0

#     while start < len(tokens):
#         end = min(start + max_len, len(tokens))
#         chunk = tokens[start:end]
#         chunks.append(chunk)
#         start += max_len - overlap

#     return chunks

# # Example usage
# tokenized_chunks = tokenizer_chunking(content, tokenizer, max_len=MAX_LEN, overlap=OVERLAP)
# print(f"Number of chunks: {len(tokenized_chunks)}")

512